In [4]:
import numpy as np
import scipy.stats as st
import random
import time
import math
from scipy.stats import multivariate_normal

In [5]:
data = np.loadtxt('seeds_dataset.txt')

In [6]:
print(data)

[[15.26   14.84    0.871  ...  2.221   5.22    1.    ]
 [14.88   14.57    0.8811 ...  1.018   4.956   1.    ]
 [14.29   14.09    0.905  ...  2.699   4.825   1.    ]
 ...
 [13.2    13.66    0.8883 ...  8.315   5.056   3.    ]
 [11.84   13.21    0.8521 ...  3.598   5.044   3.    ]
 [12.3    13.34    0.8684 ...  5.637   5.063   3.    ]]


In [7]:
features = data[:,:7]

In [8]:
print(features)

[[15.26   14.84    0.871  ...  3.312   2.221   5.22  ]
 [14.88   14.57    0.8811 ...  3.333   1.018   4.956 ]
 [14.29   14.09    0.905  ...  3.337   2.699   4.825 ]
 ...
 [13.2    13.66    0.8883 ...  3.232   8.315   5.056 ]
 [11.84   13.21    0.8521 ...  2.836   3.598   5.044 ]
 [12.3    13.34    0.8684 ...  2.974   5.637   5.063 ]]


In [9]:
# Euclidean distance
def get_distance(vec1,vec2):
    return np.sqrt(sum(np.power(vec2-vec1,2)))

# initialize centroids with random samples
def initialize(data, k):
    num,dimension = data.shape
    centroids = np.ones((k,dimension))
    for i in range(k):
        index = int(random.uniform(0,num))
        centroids[i,:] = data[index,:]
    return centroids

# K-means
def kmeans(data,k=3):
    iteration=1
    num = data.shape[0]
    # first column stores which cluster this sample belongs to,second column stores the distance**2 between this sample and its centroid
    array = np.zeros((num,2))
    for i in range(num):
        for j in range(2):
            array[i][j] = 3
    clusterInfo = np.mat(array)
    is_changed = True

    # initialize centroids
    centroids = initialize(data,k)

    while is_changed:
        is_changed = False
        # for each sample
        for i in range(num):
            min_distance = 10000000
            min_index = 0
            # find the closest centroid
            for j in range(k):
                distance = get_distance(centroids[j,:],data[i,:])
                if distance < min_distance:
                    min_distance = distance
                    min_index = j

            # update cluster
            if clusterInfo[i,0] != min_index:
                is_changed = True
                clusterInfo[i,:] = min_index, min_distance**2

        # update centroids
        for j in range(k):
            points = data[np.nonzero(clusterInfo[:,0].A==j)[0]]
            centroids[j,:] = np.mean(points,axis=0)
        iteration+=1
    
    clustering0 = []
    for i in range(num):
        clustering0.append(clusterInfo[i,0])
    clustering = np.array(clustering0)
    
    return iteration,centroids,clustering,clusterInfo

In [10]:
iteration,centroids,clustering,clusterInfo = kmeans(features,3)

In [11]:
print(iteration)
print(centroids)
print(clustering)

6
[[14.81910448 14.53716418  0.88052239  5.59101493  3.29935821  2.70658507
   5.21753731]
 [11.98865854 13.28439024  0.85273659  5.22742683  2.88008537  4.58392683
   5.0742439 ]
 [18.72180328 16.29737705  0.88508689  6.20893443  3.72267213  3.60359016
   6.06609836]]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1.
 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 0. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 0. 2. 0. 2. 2. 2. 2. 2. 2. 2. 0. 0. 0. 0. 2. 0. 0. 0. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [32]:
# accelerated K-means with triangle-inequality
# compute the distance between different centers
def centroids_dist(centroids):
    k = len(centroids)
    distance_mat = np.ones((k,k))  #distance_mat is a k*k matrix, and distance_mat[i][j] means the distance between center i and center j
    for i in range(k):
        for j in range(k):
            if i==j:
                distance_mat[i][j] = 1000000  #when computing s(c), automatically satisfy c'!=c
            distance_mat[i][j] = get_distance(centroids[i],centroids[j])
    return distance_mat

# initialize the accelerate kmeans centers, using lemma1 to avoid redundant distance calculations
def initialize_acc_k_means(data,centroids,distance_mat):
    k = len(centroids)
    num = len(data)
    lb = np.zeros((num,k)) #initially set lower bound to 0 for each point x and center c
    ub = np.zeros((num)) #initially set upper bound to 0 for each point x
    clustering = np.zeros(num)
    for i in range(num):
        dxc = get_distance(data[i],centroids[0]) #d(x,c), first in clustering 0, so this is the distance to center 0
        clustering[i] = 0
        for index in range(1,k):
            if distance_mat[0,index]>=2*dxc:  # use lemma1 here, if d(C,C')>=2d(x,c), then do not need to compute d(x,c')
                continue
            else:
                dxc_index = get_distance(data[i],centroids[index]) #otherwise, need to compute d(x,c')
                lb[i][index] = dxc_index # each time d(x,c') is computed, set l(x,c') = d(x,c')
                if dxc_index<dxc:
                    dxc = dxc_index
                    clustering[i] = index
        ub[i] = dxc #assign upper bound u(x)=min(d(x,c))
    return clustering,lb,ub

# update the clustering
def assignment(data,clustering,centroids,distance_mat,lb,ub):
    k = len(centroids)
    num = len(data)
    need_assign = np.ones(num)  #whether need re-assignment, 1 means need, 0 means no need
    sc = np.zeros(k)  # sc = 0.5*min_(c!=c')(d(c,c'))
    for i in range(k):  # for all centers c, compute sc.
        sc[i] = 0.5*np.min(distance_mat[i])
    for i in range(num):  # identify all points x such that u(x)<=s(c(x))
        cluster = int(clustering[i])  #c(x)
        ux = ub[i]  #u(x)
        if ux <= sc[cluster]: #if u(x)<=s(c(x)), no need to re-assignment
            need_assign[i] = 0
    # for all remaining points x and center c
    for i in range(num):
        cluster = int(clustering[i])
        ux = ub[i]
        if need_assign[i]==0:
            dxcx = get_distance(data[i],centroids[cluster])  #compute d(x,c(x))
            need_assign[i] = 1  
        else:
            dxcx = ub[i]  #otherwise, d(x,c(x))=u(x)
        for index in range(k):
            if index==cluster:
                continue
            if (dxcx>lb[i][index]) or (dxcx>0.5*distance_mat[cluster][index]): #d(x,c(x))>l(x,c) or d(x,c(x))>0.5*d(c(x),c)
                dxc = get_distance(data[i],centroids[index])
                if dxc<dxcx:
                    dxcx = dxc
                    clustering[i] = index
    return clustering

# update the centroid
def update(data,clustering,centroids,lb,ub):
    k = len(centroids)
    num = len(data)
    mc = np.zeros((k,data.shape[1])) # the mean of the points assigned to center c
    for i in range(k):
        mc[i] = np.mean(data[clustering==i],axis=0)
        # for each point x and center c, assign l(x,c)=max{l(x,c)-d(c,m(c)),0}
        for j in range(num):
            if lb[j][i]>=get_distance(centroids[i],mc[i]):
                lb[j][i]=lb[j][i]-get_distance(centroids[i],mc[i])
            else:
                lb[j][i]=0
        # for each point x, assign u(x) = u(x)-d(m(c(x)),c(x))
        for j in range(num):
            cluster = int(clustering[j])
            ub[j] = ub[j] - get_distance(centroids[cluster],mc[cluster])
    return mc,lb,ub

def acc_kmeans(data,initialization,k=3):
    num = len(data)
    iteration = 1
    centroids = initialization
    centroidsPrevious = initialization+1
    mc = np.zeros((k,data.shape[1]))
    c_dist = centroids_dist(centroids)
    clusterPrevious,lb,ub = initialize_acc_k_means(data,centroids,c_dist)
    while True:
        clusterCurrent = assignment(data,clusterPrevious,centroids,c_dist,lb,ub)
        # when there is different between current centroids and the previous centroids.
        if (centroids!=centroidsPrevious).any():
            centroidsPrevious = centroids
            centroids,lb,up = update(data,clusterCurrent,centroids,lb,ub)
            c_dist = centroids_dist(centroids)
            iteration+=1
        else:
            break
    return iteration,centroids,clusterCurrent

In [33]:
num = len(features)
init_number = random.sample(range(0,num),3)
# the initial centroids
initialization = np.array([features[init_number[0]],features[init_number[1]],features[init_number[2]]])
acc_iteration,acc_centroids,acc_clustering = acc_kmeans(features,initialization,3)

In [34]:
print(acc_iteration)
print(acc_centroids)
print(acc_clustering)

12
[[11.96441558 13.27480519  0.8522      5.22928571  2.87292208  4.75974026
   5.08851948]
 [14.64847222 14.46041667  0.87916667  5.56377778  3.27790278  2.64893333
   5.19231944]
 [18.72180328 16.29737705  0.88508689  6.20893443  3.72267213  3.60359016
   6.06609836]]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1.
 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 1. 2. 1. 2. 2. 2. 2. 2. 2. 2. 1. 1. 1. 1. 2. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]


In [35]:
# GMM-EM
def Expectation(data,k,mu,sigma,pi):
    num = len(data)
    lst = np.zeros((num,k))
    gamma = np.zeros((num,k))
    for i in range(num):
        for j in range(k):
            lst[i][j] = pi[j]*multivariate_normal.pdf(data[i],mean=mu[j],cov=sigma[j])
        for j in range(k):
            gamma[i][j] = lst[i][j]/sum(lst[i,:])
    return gamma

def Maximization(data,k,mu,sigma,pi):
    num = len(data)
    gamma = Expectation(data,k,mu,sigma,pi)
    for i in range(k):
        mu_values = []
        sigma_values = []
        pi_values = []
        nk = sum(gamma[:,i])
        for j in range(num):
            mu_values.append(gamma[j][i]*data[j])
            covariance = np.dot((data[j]-mu[i]).reshape(-1,1),(data[j]-mu[i]).reshape(1,-1))
            covariance = covariance+np.eye(covariance.shape[0])/1000
            sigma_values.append(gamma[j][i]*covariance)
            pi_values.append(gamma[j][i])
        mu[i] = sum(mu_values)/nk
        sigma[i] = sum(sigma_values)/nk
        pi[i] = sum(pi_values)/num
    return mu,sigma,pi

def get_log_likelihood(data,k,mu,sigma,pi):
    ll = 0
    num = len(data)
    for i in range(num):
        likel=0
        for j in range(k):
            likel+=pi[j]*multivariate_normal.pdf(data[i],mean=mu[j],cov=sigma[j])
        ll+=np.log(likel)
    return ll

def GMM_EM(data,k,mu,sigma,pi):
    llNew = 100000
    llOld = 0
    iteration = 0
    while (llNew-llOld>1e-5):
        llOld = get_log_likelihood(data,k,mu,sigma,pi)
        mu,sigma,pi = Maximization(data,k,mu,sigma,pi)
        llNew = get_log_likelihood(data,k,mu,sigma,pi)
        iteration+=1
    gamma = Expectation(data,k,mu,sigma,pi)
    lst = []
    for i in range(len(data)):
        lst.append(np.argmax(gamma[i,:]))
        clustering = np.array(lst)
    return clustering,iteration

In [36]:
num = len(features)
init_number = random.sample(range(0,num),3)
initialization_mu = np.array([features[init_number[0]],features[init_number[1]],features[init_number[2]]])
initialization_sigma = np.empty((3,7,7))
for i in range(3):
    initialization_sigma[i] = (np.eye(7))
initialization_pi = np.array([1/3,1/3,1/3])
print(initialization_mu)
print(initialization_sigma)
print(initialization_pi)
gmm_clustering,gmm_iteration = GMM_EM(features,3,initialization_mu,initialization_sigma,initialization_pi)
print(gmm_iteration)
print(gmm_clustering)

[[19.46   16.5     0.8985  6.113   3.892   4.308   6.009 ]
 [11.49   13.22    0.8263  5.304   2.695   5.388   5.31  ]
 [18.76   16.2     0.8984  6.172   3.796   3.12    6.053 ]]
[[[1. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 1.]]

 [[1. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 1.]]

 [[1. 0. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 1.]]]
[0.33333333 0.33333333 0.33333333]
84
[2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 1 2 2 1 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 2 2 2 2 2 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [37]:
# Silhouette Coefficient
def sc_single(data,index,clustering,k):
    p_cluster = clustering[index]
    p = data[index]
    a = 0
    distances = []
    for i in range(k):
        cluster = data[clustering==i,:]
        if i==p_cluster:
            for j in range(len(cluster)):
                ai = get_distance(p,cluster[j])
                a+=ai
            a/=(len(cluster)-1)
        else:
            distance = 0
            for j in range(len(cluster)):
                distance+=get_distance(p,cluster[j])
            distances.append(distance/len(cluster))
    b = min(distances)
    s=(b-a)/max(a,b)
    return s

def sc(data,clustering,k):
    s=0
    for i in range(len(data)):
        s+=sc_single(data,i,clustering,k)
    return s/len(data)

In [38]:
print("Silhouette Coefficient for kmeans:",sc(features,clustering,3)) #kmeans
print("Silhouette Coefficient for acc_kmeans:",sc(features,acc_clustering,3)) #acc_kmeans
print("Silhouette Coefficient for gmm-em:",sc(features,gmm_clustering,3)) #gmm-em

Silhouette Coefficient for kmeans: 0.46813908008596955
Silhouette Coefficient for acc_kmeans: 0.4719337319126895
Silhouette Coefficient for gmm-em: 0.41971966340999006


In [39]:
# rand index
def rand_index(real_data,clustering):
    n=len(data)
    a,b,c,d=0,0,0,0
    for i in range(n):
        for j in range(n):
            if i==j:
                continue
            elif(real_data[i]==real_data[j])and(clustering[i]==clustering[j]):
                a+=1
            elif(real_data[i]!=real_data[j])and(clustering[i]!=clustering[j]):
                b+=1
            elif(real_data[i]==real_data[j])and(clustering[i]!=clustering[j]):
                c+=1
            elif(real_data[i]!=real_data[j])and(clustering[i]==clustering[j]):
                d+=1
    ri = (a+b)/(a+b+c+d)
    return ri

In [41]:
print("rand index for kmeans:",rand_index(data[:,7],clustering))
print("rand index for acc_kmeans:",rand_index(data[:,7],acc_clustering))
print("rand index for gmm-em:",rand_index(data[:,7],gmm_clustering))

rand index for kmeans: 0.8713602187286398
rand index for acc_kmeans: 0.8743677375256322
rand index for gmm-em: 0.9242196400091137


In [42]:
# sensitive
# kmeans
sc_score=[]
ri_score=[]
for i in range(20):
    iteration,centroids,clustering,clusterInfo = kmeans(features,3)
    sc_score.append(sc(features,clustering,3))
    ri_score.append(rand_index(data[:,7],clustering))
sc_var = np.var(sc_score)
ri_var = np.var(ri_score)
print("sc_var for kmeans:",sc_var)
print("ri_var for kmeans:",ri_var)

sc_var for kmeans: 3.5638471652921954e-06
ri_var for kmeans: 2.238679405280084e-06


In [45]:
# sensitive
# acc_kmeans
sc_score=[]
ri_score=[]
for i in range(20):
    num = len(features)
    init_number = random.sample(range(0,num),3)
    initialization = np.array([features[init_number[0]],features[init_number[1]],features[init_number[2]]])
    acc_iteration,acc_centroids,acc_clustering = acc_kmeans(features,initialization,3)
    sc_score.append(sc(features,acc_clustering,3))
    ri_score.append(rand_index(data[:,7],acc_clustering))
sc_var = np.var(sc_score)
ri_var = np.var(ri_score)
print("sc_var for acc_kmeans:",sc_var)
print("ri_var for acc_kmeans:",ri_var)

sc_var for acc_kmeans: 3.6926331177279848e-06
ri_var for acc_kmeans: 4.040757042879567e-06


In [46]:
# sensitive
# GMM-EM
sc_score=[]
ri_score=[]
for i in range(10):
    num = len(features)
    init_number = random.sample(range(0,num),3)
    initialization_mu = np.array([features[init_number[0]],features[init_number[1]],features[init_number[2]]])
    initialization_sigma = np.empty((3,7,7))
    for i in range(3):
        initialization_sigma[i] = (np.eye(7))
    initialization_pi = np.array([1/3,1/3,1/3])
    gmm_clustering,gmm_iteration = GMM_EM(features,3,initialization_mu,initialization_sigma,initialization_pi)
    sc_score.append(sc(features,gmm_clustering,3))
    ri_score.append(rand_index(data[:,7],gmm_clustering))
sc_var = np.var(sc_score)
ri_var = np.var(ri_score)
print("sc_var for gmm-em:",sc_var)
print("ri_var for gmm-em:",ri_var)

sc_var for gmm-em: 0.0012744445565184633
ri_var for gmm-em: 0.009305063773790617


In [55]:
# iterations and times
# kmeans
start_time = time.time()
avg_iteration = 0
for i in range(20):
    iteration,centroids,clustering,clusterInfo = kmeans(features,3)
    avg_iteration+=iteration
end_time = time.time()
avg_iteration/=20
print("avg_iteration for kmeans:",avg_iteration)
print("avg_time for kmeans:",(end_time-start_time)/20)

avg_iteration for kmeans: 9.55
avg_time for kmeans: 0.030767643451690675


In [53]:
# iterations and times
# acc_means
avg_iteration = 0
avg_time = 0
for i in range(20):
    num = len(features)
    init_number = random.sample(range(0,num),3)
    initialization = np.array([features[init_number[0]],features[init_number[1]],features[init_number[2]]])
    start_time = time.time()
    acc_iteration,acc_centroids,acc_clustering = acc_kmeans(features,initialization,3)
    end_time = time.time()
    avg_iteration+=acc_iteration
    avg_time += end_time-start_time
avg_iteration/=20
print("avg_iteration for acc_kmeans:",avg_iteration)
print("avg_time for acc_kmeans:",avg_time/20)

avg_iteration for acc_kmeans: 10.25
avg_time for acc_kmeans: 0.09971770048141479


In [50]:
# iterations and times
# gmm-em
avg_iteration = 0
avg_time = 0
for i in range(10):
    num = len(features)
    init_number = random.sample(range(0,num),3)
    initialization_mu = np.array([features[init_number[0]],features[init_number[1]],features[init_number[2]]])
    initialization_sigma = np.empty((3,7,7))
    for i in range(3):
        initialization_sigma[i] = (np.eye(7))
    initialization_pi = np.array([1/3,1/3,1/3])
    start_time = time.time()
    gmm_clustering,gmm_iteration = GMM_EM(features,3,initialization_mu,initialization_sigma,initialization_pi)
    end_time = time.time()
    avg_iteration+=gmm_iteration
    avg_time += end_time-start_time
avg_iteration/=10
print("avg_iteration for gmm-em:",avg_iteration)
print("avg_time for gmm-em:",avg_time/10)

avg_iteration for gmm-em: 38.5
avg_time for gmm-em: 7.151297473907471
